## load data

In [183]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

#train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'TIMESTAMP'])
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
'''
train_x['y0'] = 0
train_x['y1'] = 0
train_x['y2'] = 0
test_x['y0'] = 0
test_x['y1'] = 0
test_x['y2'] = 0
'''

"\ntrain_x['y0'] = 0\ntrain_x['y1'] = 0\ntrain_x['y2'] = 0\ntest_x['y0'] = 0\ntest_x['y1'] = 0\ntest_x['y2'] = 0\n"

## Preprocessing

### Label Encoder

In [184]:
from sklearn.preprocessing import LabelEncoder

qual_col = ['LINE','PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[i] = le.transform(test_x[i])

print('done')

done


In [185]:
train_x.columns

Index(['Y_Class', 'LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5',
       'X_6', 'X_7',
       ...
       'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
       'X_2873', 'X_2874', 'X_2875'],
      dtype='object', length=2878)

### split

In [186]:
## PRODUCT_CODE

# train
train_x_1 = train_x[train_x['PRODUCT_CODE'] == 0]#.drop('Y_Class', axis=1)
train_x_2 = train_x[train_x['PRODUCT_CODE'] == 1]#.drop('Y_Class', axis=1)
train_x_3 = train_x[train_x['PRODUCT_CODE'] == 2]#.drop('Y_Class', axis=1)

train_y_1 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 0]
train_y_2 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 1]
train_y_3 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 2]

print('train_x_1 shape :', train_x_1.shape,
      '\ntrain_x_2 shape :', train_x_2.shape,
      '\ntrain_x_3 shape :', train_x_3.shape)

# test
test_x_1 = test_x[test_x['PRODUCT_CODE'] == 0]
test_x_2 = test_x[test_x['PRODUCT_CODE'] == 1]
test_x_3 = test_x[test_x['PRODUCT_CODE'] == 2]

print('done')

train_x_1 shape : (249, 2878) 
train_x_2 shape : (6, 2878) 
train_x_3 shape : (343, 2878)
done


In [187]:
# LINE

## TRAIN

# line 1
train_x_1_1 = train_x_1[train_x_1['LINE'] == 0]

# line 2
train_x_1_2 = train_x_1[train_x_1['LINE'] == 1]

# line 3
train_x_1_3 = train_x_1[train_x_1['LINE'] == 2]

# line 4
train_x_1_4 = train_x_1[train_x_1['LINE'] == 3]

# line 5
train_x_2_5 = train_x_2[train_x_2['LINE'] == 4]
train_x_3_5 = train_x_3[train_x_3['LINE'] == 4]

# line 6
train_x_2_6 = train_x_2[train_x_2['LINE'] == 5]
train_x_3_6 = train_x_3[train_x_3['LINE'] == 5]


## TEST

# line 1
test_x_1_1 = test_x_1[test_x_1['LINE'] == 0]

# line 2
test_x_1_2 = test_x_1[test_x_1['LINE'] == 1]

# line 3
test_x_1_3 = test_x_1[test_x_1['LINE'] == 2]

# line 4
test_x_1_4 = test_x_1[test_x_1['LINE'] == 3]

# line 5
test_x_2_5 = test_x_2[test_x_2['LINE'] == 4]
test_x_3_5 = test_x_3[test_x_3['LINE'] == 4]

# line 6
test_x_2_6 = test_x_2[test_x_2['LINE'] == 5]
test_x_3_6 = test_x_3[test_x_3['LINE'] == 5]


print('train')
train_set = [train_x_1_1, train_x_1_2, train_x_1_3, train_x_1_4, train_x_2_5, train_x_3_5, train_x_2_6, train_x_3_6]

for set in train_set:
    print(set.shape)


print('\n\ntest')
test_set = [test_x_1_1, test_x_1_2, test_x_1_3, test_x_1_4, test_x_2_5, test_x_3_5, test_x_2_6, test_x_3_6]

for set in test_set:
    print(set.shape)


train
(59, 2878)
(70, 2878)
(78, 2878)
(42, 2878)
(3, 2878)
(172, 2878)
(3, 2878)
(171, 2878)


test
(14, 2877)
(14, 2877)
(13, 2877)
(26, 2877)
(3, 2877)
(108, 2877)
(1, 2877)
(131, 2877)


### null

In [190]:
nc = train_x_1_1.isnull().sum()
ncv = nc.values
nci = nc.index

for idx, i in enumerate(ncv):
    #print(i)
    rate = i / train_x_1_1.shape[0]
    #print(rate)
    #print(nci[idx])
    if i > 0:
        if rate > 0.30:
            #train_x_1_1.drop([nci[idx]], axis=1, inplace=True)
            train_x_1_1[nci[idx]] = np.NaN

print(train_x_1_1.shape)

(59, 2878)


In [199]:
for df in [train_set, test_set]:
    
    if 


    for set in df:

        nc = set.isnull().sum()
        ncv = nc.values
        nci = nc.index

        for idx, i in enumerate(ncv):
            #print(i)
            rate = i / set.shape[0]
            #print(rate)
            #print(nci[idx])
            if i > 0:
                if rate > 0.30:
                    #train_x_1_1.drop([nci[idx]], axis=1, inplace=True)
                    set[nci[idx]] = np.NaN

        print(set.shape)

'''    print('\n\n\n### test ###')
    for set in test_set:

        nc = set.isnull().sum()
        ncv = nc.values
        nci = nc.index

        for idx, i in enumerate(ncv):
            rate = i / set.shape[0]
            if i > 0:
                if rate > 0.30:
                    set[nci[idx]] = np.NaN

        print(set.shape)'''

### df ###
(59, 2878)
(70, 2878)
(78, 2878)
(42, 2878)
(3, 2878)
(172, 2878)
(3, 2878)
(171, 2878)
### df ###
(14, 2877)
(14, 2877)
(13, 2877)
(26, 2877)
(3, 2877)
(108, 2877)
(1, 2877)
(131, 2877)


"    print('\n\n\n### test ###')\n    for set in test_set:\n\n        nc = set.isnull().sum()\n        ncv = nc.values\n        nci = nc.index\n\n        for idx, i in enumerate(ncv):\n            rate = i / set.shape[0]\n            if i > 0:\n                if rate > 0.30:\n                    set[nci[idx]] = np.NaN\n\n        print(set.shape)"

In [194]:
train_x = pd.concat(train_set, axis=0).sort_index()
test_x = pd.concat(test_set, axis=0).sort_index()

train_x.drop('Y_Class', axis=1, inplace=True)

print('train\n',train_x.shape)
print('test\n',test_x.shape)

train
 (598, 2877)
test
 (310, 2877)


In [196]:
from sklearn.model_selection import train_test_split

x_t, x_v, y_t, y_v = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [197]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(x_t, y_t)
pred = xgb.predict(x_v)

from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(pred, y_v))
print(confusion_matrix(pred, y_v))

0.7166666666666667
[[ 9  1  2]
 [14 71 13]
 [ 0  4  6]]


In [39]:
print('train :', train_x.shape)
print('test :', test_x.shape)

train : (598, 2880)
test : (310, 2880)


In [40]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)


submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = pred
submit_csv.to_csv('XGBoost_null_per.csv', index=False)